<a href="https://colab.research.google.com/github/LordHarsh/Neural_Style_Transfer/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lets install the package

##### This is a package I made to perform Neural Style Transfer on video frames

In [ ]:
!pip install StyleTransferTensorFlow==0.1.3

In [ ]:
import StyleTransferTensorFlow as st
import os

Now we need two thing-


*   The video on which we need to perform style transfer- For best results try taking video of a scenry, building or city.
*   The style image- This needs to an art image. This program tries to transfer artistic features of this image to frames of the video.



Examples for video-

In [ ]:
#@title Select the video- (Some examples are given below)

video_input = "YouTube URL" #@param ["Upload from PC", "YouTube URL"]
video_url = "Empire State Building" #@param ["Leave empty if uploading from PC", "Empire State Building"] {allow-input: true}
#@markdown Copy and paste url of the picture here.
!pip install pytube

from pytube import YouTube

def YTDownload(enterlink):
  return YouTube(enterlink).streams.filter(only_video=True, subtype='mp4').order_by('resolution').desc().first().download()

vidpath = ''
if video_input=="YouTube URL":
  if video_url=="Empire State Building":
    video_url='https://www.youtube.com/watch?v=PfOqGpL3jlk'
  vidpath = YTDownload(video_url)
  print("Video Downloaded.....")

elif video_input=="Upload from PC":

  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    vidpath=os.path.join(os.getcwd(), fn)
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
#@title Select the style picture- (Some examples are given below)
pic_input_type = "URL" #@param ["Upload from PC", "URL"]
pic_url = "Starry Night" #@param ["Leave empty if uploading from PC", "Starry Night", "Custom"] {allow-input: true}
#@markdown Copy and paste url of the picture here.
picpath = ''
if pic_input_type=="URL":
  if pic_url=="Starry Night":
    pic_url='https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1200px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'
  import requests
  picpath=os.path.join(os.getcwd(), "style.jpg")
  with open('style.jpg', 'wb') as handle:
          response = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1200px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', stream=True)

          if not response.ok:
              print (response)

          for block in response.iter_content(1024):
              if not block:
                  break
              handle.write(block)
  print("Picture Downloaded.....")
elif pic_input_type=="Upload from PC":

  from google.colab import files

  uploaded = files.upload()

  for fn in uploaded.keys():
    picpath=os.path.join(os.getcwd(), fn)
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

### -> Now this cell given below do all the main task- extracting frames from video, applying style transfer to each frame, converting frames to cv2 images and converting this the video.

In [ ]:
mp4_video_path = st.make.MakeVideo(vidpath, picpath)   

In [ ]:
#@title View the new video directly in the browser through Colab 

#@markdown While Running this cell, it will ask "Enter verification code". This code is present in the link which is present just above the text box. It will read something as "https://accounts.google.com/o/........" This will ask you to allow some access, you can do it. Don't worry its safe. But if you dont want to do it run the last cell of this note book, it will download the video directly to your PC.

!pip install -U kora

from kora.drive import upload_public

url = upload_public(mp4_video_path)

# then display it

from IPython.display import HTML

HTML(f"""<video src={url} width=500 controls/>""")

In [ ]:
#@title Lets do some custum video making
invert_image_colour = False #@param ["True", "False"] {type:"raw"}
#@markdown * Try changing this value to True from False
extract_frame_rate = 30 #@param {type:"slider", min:1, max:40, step:1}
#@markdown * Try changing this value- Setting this to i means that every ith frame is included in the new style video.
#@markdown * So, extract_frame_rate=1 means every frame of orignal video is included in the new video and extract_frame_rate mean every 40th frame is included in the video.
#@markdown * Note -> The lower the value of extract_frame_rate, the more computation time it will take.

mp4_video_path = st.make.MakeVideo(vidpath, picpath, invert=invert_image_colour,  extractframerate=extract_frame_rate)

In [ ]:
#@title View the new video directly in the browser through Colab 

#@markdown While Running this cell, it will ask "Enter verification code". This code is present in the link which is present just above the text box. It will read something as "https://accounts.google.com/o/........" This will ask you to allow some access, you can do it. Don't worry its safe. But if you dont want to do it run the last cell of this note book, it will download the video directly to your PC.

!pip install -U kora

from kora.drive import upload_public

url = upload_public(mp4_video_path)

# then display it

from IPython.display import HTML

HTML(f"""<video src={url} width=500 controls/>""")

While Running this cell, it will ask "Enter verification code". This code is present in the link which is present just above the text box. It will read something as "https://accounts.google.com/o/........" This will ask you to allow some access, you can do it. Don't worry its safe.
But if you dont want to do it run the last cell of this note book, it will download the video directly to your PC.

In [ ]:
from google.colab import files
files.download(mp4_video_path)